SELECTION DU MODELE

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:

# Download the dataset
dataset = pd.read_csv('dataset_cleaned.csv')

# Display the first few rows of the dataframe
dataset.head()

,nom_region,nom_espece,superficie,pluviometrie,rendement_moyen,annee,temperature_moyenne,mois_plantation,log_pluviometrie,categorie_pluviometrie
0,Abidjan,Cacao,311.35,221.88,9.40,2019,25.670,4,5.406634,Moyenne
1,Abidjan,Cacao,141.35,221.88,9.40,2019,25.670,8,5.406634,Moyenne
2,Abidjan,Cacao,301.74,221.88,9.40,2019,25.670,3,5.406634,Moyenne
3,Abidjan,Café,114.35,92.11,6.29,2019,29.375,6,4.533782,Faible
4,Abidjan,Café,253.65,92.11,6.29,2019,29.375,12,4.533782,Faible


Preprocessing

In [10]:
import numpy as np
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler



In [11]:
# 1. Séparation des caractéristiques (features) et de la cible (target)
X = dataset.drop('rendement_moyen', axis=1)
y = dataset['rendement_moyen']

# 2. Identification des colonnes numériques et non numériques
numeric_columns = X.select_dtypes(include=['number']).columns
categorical_columns = X.select_dtypes(exclude=['number']).columns

print("Colonnes numériques :", numeric_columns)
print("Colonnes catégoriques :", categorical_columns)

# 3. Création du préprocesseur
#    - Encodage OneHotEncoder pour les colonnes catégoriques.
#    - Standardisation (StandardScaler) pour les colonnes numériques.
preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),  # Standardisation des colonnes numériques
    (OneHotEncoder(), make_column_selector(dtype_exclude=np.number))   # Encodage des colonnes catégoriques
)

# 4. Création du pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# 5. Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Application du pipeline aux données
X_train = pipeline.fit_transform(X_train)  # Ajustement et transformation sur l'ensemble d'entraînement
X_test = pipeline.transform(X_test)        # Transformation uniquement sur l'ensemble de test

Colonnes numériques : Index(['superficie', 'pluviometrie', 'annee', 'temperature_moyenne',
       'mois_plantation', 'log_pluviometrie'],
      dtype='object')
Colonnes catégoriques : Index(['nom_region', 'nom_espece', 'categorie_pluviometrie'], dtype='object')


RFECV pour Validation Croisée Automatique

In [16]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV



In [17]:
# Modèle utilisé pour RFECV
model = RandomForestRegressor(random_state=42)

# RFECV pour sélection récursive avec validation croisée
rfecv = RFECV(estimator=model, step=1, cv=5, scoring='r2')  # 5-fold cross-validation
rfecv.fit(X_train, y_train)

# Résultats
print("Nombre optimal de caractéristiques :", rfecv.n_features_)

Nombre optimal de caractéristiques : 35


AttributeError: 'RFECV' object has no attribute 'grid_scores_'

In [18]:
mean_cv_scores = rfecv.cv_results_['mean_test_score']
print("Scores moyens (R²) pour chaque nombre de caractéristiques :", mean_cv_scores)


Scores moyens (R²) pour chaque nombre de caractéristiques : [0.76783509 0.77201804 0.77213856 0.61083419 0.55879801 0.61838578
 0.6436908  0.67219025 0.71408638 0.72905949 0.72760366 0.74154994
 0.73986543 0.75987806 0.76308018 0.77606944 0.77916442 0.7819086
 0.78764687 0.79147297 0.79826013 0.80286214 0.80654083 0.80619448
 0.80809024 0.81293046 0.81163412 0.81400403 0.8142673  0.81782658
 0.81991561 0.82198333 0.82315639 0.8260233  0.82700429 0.82607352
 0.82435543]
